In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
class ConcreteStrength(nn.Module):
        def __init__(self, input_dim):
            super(ConcreteStrength, self).__init__()
            self.model = nn.Sequential(
                nn.Linear(input_dim, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.Linear(32, 1)  # Output layer
            )
    
        def forward(self, x):
            return self.model(x)
    

In [5]:
model = torch.load("Model.pth")

In [7]:
scaler_params = {
    "mean_": [
        283.36080097087375,
        2.5552736017613937,
        2.091743575425426,
        181.79708737864075,
        1.4479760883048582,
        973.5485436893204,
        772.1074029126214,
        3.239097061274715
    ],
    "scale_": [
        107.47113139056532,
        2.4137541776391926,
        2.3698221380244933,
        21.308962544444068,
        1.1534634219125226,
        78.646864026356,
        80.93556052399222,
        1.1001286591127104
    ],
    "var_": [
        11550.044082368155,
        5.826209230070655,
        5.616056965870981,
        454.07188472052025,
        1.3304778656901461,
        6185.329221180129,
        6550.564957332807,
        1.21028306660113
    ],
    "n_samples_seen_": 824,
    "feature_names_in_": [
        "Cement",
        "BlastFurnaceSlag",
        "FlyAsh",
        "Water",
        "Superplasticizer",
        "CoarseAggregate",
        "FineAggregate",
        "Age"
    ]
}
scaler = StandardScaler()
scaler.mean_ = np.array(scaler_params["mean_"])
scaler.scale_ = np.array(scaler_params["scale_"])
scaler.var_ = np.array(scaler_params["var_"])
scaler.n_samples_seen_ = scaler_params["n_samples_seen_"]
scaler.feature_names_in_ = np.array(scaler_params["feature_names_in_"])

print("Scaler restored successfully!")

Scaler restored successfully!


In [9]:
print(model)

ConcreteStrength(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [11]:
test_data_path = "Concrete_Data_Train.csv" # provide test dataset path
df_test = pd.read_csv(test_data_path)

df_test.columns = ['Cement','BlastFurnaceSlag','FlyAsh','Water','Superplasticizer','CoarseAggregate','FineAggregate','Age','ConcreteCompressiveStrength']

In [13]:
X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

In [15]:
X_test["BlastFurnaceSlag"] = np.log1p(X_test["BlastFurnaceSlag"])
X_test["FlyAsh"] = np.log1p(X_test["FlyAsh"])
X_test["Superplasticizer"] = np.log1p(X_test["Superplasticizer"])
X_test["Age"] = np.log1p(X_test["Age"])

X_test_scaled = scaler.transform(X_test)

In [17]:
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [19]:
model.eval()
with torch.no_grad():
        y_pred = model(X_test_tensor)

# predicted values
y_pred_np = y_pred.numpy().flatten()

r2score = r2_score(y_test, y_pred_np)
mse = mean_squared_error(y_test, y_pred_np)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_np)

In [21]:
print(f"R² Score: {r2score:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

R² Score: 0.9839
Root Mean Squared Error (RMSE): 2.1362
Mean Squared Error (MSE): 4.5632
Mean Absolute Error (MAE): 1.5502
